In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [23]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [24]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [25]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [26]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [27]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [28]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [29]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [30]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [31]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [32]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [33]:
community = 'hill_district'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [34]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1161.8
Electric Clothes Dryer                                              1304.7
Induction Cooking                                                   1997.8
Heat Pump Clothes Dryer                                             2625.4
Basic Enclosure                                                     2797.9
Heat Pump Water Heater                                              3034.4
Enhanced Enclosure                                                  3613.7
Mininum Efficiency Heat Pump with Existing Heat Backup             15041.3
Mininum Efficiency Heat Pump with Electric Heat Backup             15077.4
Mininum Efficiency Whole Home Electrification                      16406.7
High Efficiency Heat Pump with Electric Heat Backup                21726.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25129.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25510.7
High Efficie

## 1.2 Total Annual Savings

In [35]:
figure_total_savings_1(community_results)

In [36]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [37]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [38]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Whole Home Electrification                      85359.314126
Mininum Efficiency Heat Pump with Electric Heat Backup             99857.810409
Electric Clothes Dryer                                             17948.299257
Electric Cooking                                                   55802.894052
Induction Cooking                                                  97806.576208
Mininum Efficiency Heat Pump with Existing Heat Backup             97899.814126
Heat Pump Clothes Dryer                                            53332.089219
Heat Pump Water Heater                                             99158.526022
High Efficiency Heat Pump with Electric Heat Backup                99857.810409
High Efficiency Whole Home Electrification                         99857.810409
Basic Enclosure                                                    97107.291822
Enhanced Enclosure                                                 98086.289963
Basic Enclosure + High Effi

In [39]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,2142,99857.810409,64.357087,58.1600,73.1900
Basic Enclosure + High Efficiency Whole Home Electrification,2142,99857.810409,63.826793,57.5700,72.4300
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,2142,99857.810409,62.355327,55.0300,71.6700
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,2142,99857.810409,61.825079,54.6200,71.0000
High Efficiency Whole Home Electrification,2142,99857.810409,58.460976,52.7500,65.8200
Mininum Efficiency Whole Home Electrification,1831,85359.314126,41.910442,36.2350,53.2275
High Efficiency Heat Pump with Electric Heat Backup,2142,99857.810409,40.160196,30.6000,52.4200
Mininum Efficiency Heat Pump with Electric Heat Backup,2142,99857.810409,27.503870,18.3000,38.1900
Enhanced Enclosure,2104,98086.289963,22.545813,15.2850,29.8700


# 2. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Single-family detached, renter, pre 1940, masonry construction, 0-80% AMI

Segment 2. Single-family attached, renter, pre 1940, masonry construction, 0-80% AMI

Segment 3. Multi-family, masonry construction, 0-80% AMI

Segment 4. 0-80% AMI

In [40]:
income = '0-80%AMI'

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_construction = ['Brick', 'Concrete']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Single-Family Attached']
seg2_vintage = ['<1940']
seg2_tenure = ['Renter']
seg2_construction = ['Brick', 'Concrete']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units']
seg3_construction = ['Brick', 'Concrete']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &  
                     community_results['build_existing_model.geometry_wall_type'].isin(seg1_construction) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) &   
                     community_results['build_existing_model.geometry_wall_type'].isin(seg2_construction) &
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.geometry_wall_type'].isin(seg3_construction) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
seg4 = community_results.copy().assign(segment='segment4')
seg4 = seg4.loc[seg4['build_existing_model.area_median_income'].isin(seg4_median_income)]
community_results = pd.concat([community_results, seg4], axis=0)

In [41]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: segment1: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,239,11141.931227,70.508494,67.5050,76.8800
Basic Enclosure + High Efficiency Whole Home Electrification,239,11141.931227,70.131799,66.9225,76.6350
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,239,11141.931227,69.013305,65.5475,75.6200
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,239,11141.931227,68.636527,65.0850,75.4025
High Efficiency Whole Home Electrification,239,11141.931227,61.911130,58.3700,70.5950
High Efficiency Heat Pump with Electric Heat Backup,239,11141.931227,50.200544,45.1975,60.6875
Mininum Efficiency Whole Home Electrification,238,11095.312268,45.900504,41.7300,59.2500
Mininum Efficiency Heat Pump with Electric Heat Backup,239,11141.931227,35.725021,30.1750,48.7600
Enhanced Enclosure,239,11141.931227,31.723849,27.9700,35.8475


Summary for segment: segment2: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,354,16503.111524,69.804350,63.8600,76.7600
Basic Enclosure + High Efficiency Whole Home Electrification,354,16503.111524,69.487486,63.8600,76.3300
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,354,16503.111524,67.701243,60.9300,75.2900
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,354,16503.111524,67.384435,60.9300,74.6900
High Efficiency Whole Home Electrification,354,16503.111524,63.417797,57.7100,70.6400
Mininum Efficiency Whole Home Electrification,336,15663.970260,49.566399,43.1100,58.0100
High Efficiency Heat Pump with Electric Heat Backup,354,16503.111524,47.680989,39.0400,57.4300
Mininum Efficiency Heat Pump with Electric Heat Backup,354,16503.111524,35.272316,25.9800,44.4500
Enhanced Enclosure,354,16503.111524,28.409350,23.5200,32.8600


Summary for segment: segment3: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1549,72212.767658,62.163079,55.2950,70.9825
Basic Enclosure + High Efficiency Whole Home Electrification,1549,72212.767658,61.560310,54.9850,70.0950
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1549,72212.767658,60.106320,52.6625,69.2900
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1549,72212.767658,59.503615,52.1000,68.3425
High Efficiency Whole Home Electrification,1549,72212.767658,56.795836,50.8775,64.3225
Mininum Efficiency Whole Home Electrification,1257,58600.031599,39.108504,29.8225,50.9700
High Efficiency Heat Pump with Electric Heat Backup,1549,72212.767658,36.892279,26.4675,48.8100
Mininum Efficiency Heat Pump with Electric Heat Backup,1549,72212.767658,24.460045,15.3675,35.1225
Enhanced Enclosure,1511,70441.247212,19.720371,12.5500,26.1400


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,2142,99857.810409,64.357087,58.1600,73.1900
Basic Enclosure + High Efficiency Whole Home Electrification,2142,99857.810409,63.826793,57.5700,72.4300
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,2142,99857.810409,62.355327,55.0300,71.6700
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,2142,99857.810409,61.825079,54.6200,71.0000
High Efficiency Whole Home Electrification,2142,99857.810409,58.460976,52.7500,65.8200
Mininum Efficiency Whole Home Electrification,1831,85359.314126,41.910442,36.2350,53.2275
High Efficiency Heat Pump with Electric Heat Backup,2142,99857.810409,40.160196,30.6000,52.4200
Mininum Efficiency Heat Pump with Electric Heat Backup,2142,99857.810409,27.503870,18.3000,38.1900
Enhanced Enclosure,2104,98086.289963,22.545813,15.2850,29.8700


## 2.1 Upgrade Cost Per Dwelling Unit 

In [42]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1161.8
Electric Clothes Dryer                                              1304.7
Induction Cooking                                                   1997.8
Heat Pump Clothes Dryer                                             2625.4
Heat Pump Water Heater                                              3195.5
Basic Enclosure                                                     7206.9
Enhanced Enclosure                                                  9124.7
Mininum Efficiency Heat Pump with Existing Heat Backup             22682.9
Mininum Efficiency Heat Pump with Electric Heat Backup             22805.0
Mininum Efficiency Whole Home Electrification                      26571.8
High Efficiency Heat Pump with Electric Heat Backup                29144.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34719.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35191.1
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1161.8
Electric Clothes Dryer                                              1304.7
Induction Cooking                                                   1997.8
Heat Pump Clothes Dryer                                             2625.4
Heat Pump Water Heater                                              3048.2
Basic Enclosure                                                     4064.7
Enhanced Enclosure                                                  4941.6
Mininum Efficiency Heat Pump with Existing Heat Backup             18907.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18949.5
Mininum Efficiency Whole Home Electrification                      22019.2
High Efficiency Heat Pump with Electric Heat Backup                24276.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        27769.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27945.4
High Efficie

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1161.8
Electric Clothes Dryer                                              1304.7
Basic Enclosure                                                     1789.7
Induction Cooking                                                   1997.8
Enhanced Enclosure                                                  2430.9
Heat Pump Clothes Dryer                                             2625.4
Heat Pump Water Heater                                              3006.2
Mininum Efficiency Heat Pump with Existing Heat Backup             12963.9
Mininum Efficiency Whole Home Electrification                      12981.8
Mininum Efficiency Heat Pump with Electric Heat Backup             13000.2
High Efficiency Heat Pump with Electric Heat Backup                19999.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        23046.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     23460.7
High Efficie

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1161.8
Electric Clothes Dryer                                              1304.7
Induction Cooking                                                   1997.8
Heat Pump Clothes Dryer                                             2625.4
Basic Enclosure                                                     2797.9
Heat Pump Water Heater                                              3034.4
Enhanced Enclosure                                                  3613.7
Mininum Efficiency Heat Pump with Existing Heat Backup             15041.3
Mininum Efficiency Heat Pump with Electric Heat Backup             15077.4
Mininum Efficiency Whole Home Electrification                      16406.7
High Efficiency Heat Pump with Electric Heat Backup                21726.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25129.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25510.7
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [43]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.05984
Enhanced Enclosure                                                 0.05466
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01108
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01038
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00985
Basic Enclosure + High Efficiency Whole Home Electrification       0.00922
Heat Pump Clothes Dryer                                            0.00611
Heat Pump Water Heater                                             0.00401
High Efficiency Whole Home Electrification                        -0.00596
Induction Cooking                                                 -0.00948
High Efficiency Heat Pump with Electric Heat Backup               -0.01062
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.01322
Electric Cooking                                                  -0.03513
Electric Clo

segment2


upgrade_name
Basic Enclosure                                                    0.08015
Enhanced Enclosure                                                 0.07501
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01510
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01451
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01312
Basic Enclosure + High Efficiency Whole Home Electrification       0.01259
Heat Pump Clothes Dryer                                            0.00801
Heat Pump Water Heater                                             0.00400
High Efficiency Whole Home Electrification                         0.00228
High Efficiency Heat Pump with Electric Heat Backup               -0.00095
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00888
Induction Cooking                                                 -0.01091
Electric Cooking                                                  -0.03365
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.12725
Enhanced Enclosure                                                 0.10620
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02240
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02186
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01976
Basic Enclosure + High Efficiency Whole Home Electrification       0.01924
High Efficiency Whole Home Electrification                         0.01305
High Efficiency Heat Pump with Electric Heat Backup                0.01246
Heat Pump Water Heater                                             0.01146
Heat Pump Clothes Dryer                                            0.00846
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00532
Induction Cooking                                                 -0.00733
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00960
Mininum Effi

segment4


upgrade_name
Basic Enclosure                                                    0.09570
Enhanced Enclosure                                                 0.08424
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01933
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01875
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01701
Basic Enclosure + High Efficiency Whole Home Electrification       0.01647
Heat Pump Water Heater                                             0.00934
High Efficiency Whole Home Electrification                         0.00827
Heat Pump Clothes Dryer                                            0.00788
High Efficiency Heat Pump with Electric Heat Backup                0.00653
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00078
Induction Cooking                                                 -0.00817
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.02287
Electric Coo

 ### Annual Energy Savings Per Dollar Invested

In [44]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00519
Enhanced Enclosure                                                 0.00467
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00270
Heat Pump Water Heater                                             0.00269
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00268
Mininum Efficiency Whole Home Electrification                      0.00246
Basic Enclosure + High Efficiency Whole Home Electrification       0.00244
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00243
High Efficiency Heat Pump with Electric Heat Backup                0.00242
High Efficiency Whole Home Electrification                         0.00231
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00229
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00137
Electric Cooking                                                   0.00118
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.00594
Enhanced Enclosure                                                 0.00552
Heat Pump Water Heater                                             0.00241
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00231
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00231
Mininum Efficiency Whole Home Electrification                      0.00216
Basic Enclosure + High Efficiency Whole Home Electrification       0.00206
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00206
High Efficiency Heat Pump with Electric Heat Backup                0.00193
High Efficiency Whole Home Electrification                         0.00189
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00186
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00119
Electric Cooking                                                   0.00113
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.00596
Enhanced Enclosure                                                 0.00497
Heat Pump Water Heater                                             0.00196
Mininum Efficiency Whole Home Electrification                      0.00189
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00148
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00147
Basic Enclosure + High Efficiency Whole Home Electrification       0.00135
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00135
High Efficiency Whole Home Electrification                         0.00124
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00119
High Efficiency Heat Pump with Electric Heat Backup                0.00111
Electric Cooking                                                   0.00108
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00084
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.00573
Enhanced Enclosure                                                 0.00501
Heat Pump Water Heater                                             0.00212
Mininum Efficiency Whole Home Electrification                      0.00208
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00182
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00181
Basic Enclosure + High Efficiency Whole Home Electrification       0.00165
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00164
High Efficiency Whole Home Electrification                         0.00152
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00151
High Efficiency Heat Pump with Electric Heat Backup                0.00146
Electric Cooking                                                   0.00110
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00100
Induction Co

 ### Annual Emission Savings Per Dollar Invested

In [45]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.35597
Enhanced Enclosure                                                 0.32133
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.16401
Heat Pump Water Heater                                             0.16387
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.16355
Basic Enclosure + High Efficiency Whole Home Electrification       0.14840
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.14817
High Efficiency Heat Pump with Electric Heat Backup                0.12591
High Efficiency Whole Home Electrification                         0.12557
Mininum Efficiency Whole Home Electrification                      0.09222
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07346
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06788
Electric Cooking                                                   0.04979
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.41920
Enhanced Enclosure                                                 0.39011
Heat Pump Water Heater                                             0.14827
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.14708
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14658
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13073
Basic Enclosure + High Efficiency Whole Home Electrification       0.13021
High Efficiency Whole Home Electrification                         0.11057
High Efficiency Heat Pump with Electric Heat Backup                0.10829
Mininum Efficiency Whole Home Electrification                      0.09115
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07168
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06168
Electric Cooking                                                   0.04705
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.46600
Enhanced Enclosure                                                 0.38825
Heat Pump Water Heater                                             0.12796
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.10630
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.10630
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.09675
Basic Enclosure + High Efficiency Whole Home Electrification       0.09661
High Efficiency Whole Home Electrification                         0.08399
Mininum Efficiency Whole Home Electrification                      0.08025
High Efficiency Heat Pump with Electric Heat Backup                0.07517
Mininum Efficiency Heat Pump with Electric Heat Backup             0.05823
Mininum Efficiency Heat Pump with Existing Heat Backup             0.05463
Electric Cooking                                                   0.04551
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.42193
Enhanced Enclosure                                                 0.36948
Heat Pump Water Heater                                             0.13558
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.12255
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12249
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11090
Basic Enclosure + High Efficiency Whole Home Electrification       0.11080
High Efficiency Whole Home Electrification                         0.09501
High Efficiency Heat Pump with Electric Heat Backup                0.08888
Mininum Efficiency Whole Home Electrification                      0.08546
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06360
Mininum Efficiency Heat Pump with Existing Heat Backup             0.05833
Electric Cooking                                                   0.04635
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [46]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure                                                  5557245.2
Basic Enclosure                                                     4804983.3
Enhanced Enclosure + High Efficiency Whole Home Electrification     4347582.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      4344716.6
Basic Enclosure + High Efficiency Whole Home Electrification        4018572.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         4015706.6
Heat Pump Clothes Dryer                                              167474.4
Heat Pump Water Heater                                               142851.3
Induction Cooking                                                   -206705.5
Electric Cooking                                                    -275885.4
Electric Clothes Dryer                                              -297335.7
High Efficiency Whole Home Electrification                         -2439657.1
Mininum Efficiency Heat Pump with Existing Heat Bac

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification     6985287.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      6965604.2
Basic Enclosure + High Efficiency Whole Home Electrification        6667805.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         6648121.8
Enhanced Enclosure                                                  6117542.2
Basic Enclosure                                                     5376572.9
High Efficiency Whole Home Electrification                          1190217.6
Heat Pump Clothes Dryer                                              309969.1
Heat Pump Water Heater                                               200117.0
Electric Clothes Dryer                                              -288357.7
Induction Cooking                                                   -355536.1
High Efficiency Heat Pump with Electric Heat Backup                 -381998.1
Electric Cooking                                   

segment3


upgrade_name
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     37941296.9
Enhanced Enclosure + High Efficiency Whole Home Electrification    37717775.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        36377657.8
Basic Enclosure + High Efficiency Whole Home Electrification       36154136.5
High Efficiency Whole Home Electrification                         24456332.4
Enhanced Enclosure                                                 18185691.7
High Efficiency Heat Pump with Electric Heat Backup                17990594.6
Basic Enclosure                                                    15818981.4
Mininum Efficiency Heat Pump with Existing Heat Backup              4864649.1
Heat Pump Water Heater                                              2467069.5
Heat Pump Clothes Dryer                                              625455.7
Electric Clothes Dryer                                              -483274.3
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     49251617.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    49050645.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        47041486.3
Basic Enclosure + High Efficiency Whole Home Electrification       46840513.9
Enhanced Enclosure                                                 29860479.1
Basic Enclosure                                                    26000537.5
High Efficiency Whole Home Electrification                         23206893.0
High Efficiency Heat Pump with Electric Heat Backup                14161241.6
Heat Pump Water Heater                                              2810037.7
Heat Pump Clothes Dryer                                             1102899.2
Electric Clothes Dryer                                             -1068967.7
Mininum Efficiency Heat Pump with Existing Heat Backup             -1155076.7
Induction Cooking                                  

### Annual Total Energy Savings

In [47]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1070778.2
Basic Enclosure + High Efficiency Whole Home Electrification       1064780.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1049676.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1043677.8
High Efficiency Whole Home Electrification                          947039.1
High Efficiency Heat Pump with Electric Heat Backup                 784931.8
Mininum Efficiency Whole Home Electrification                       724501.4
Mininum Efficiency Heat Pump with Electric Heat Backup              583025.8
Enhanced Enclosure                                                  474982.3
Basic Enclosure                                                     416709.5
Mininum Efficiency Heat Pump with Existing Heat Backup              336812.8
Heat Pump Water Heater                                               95737.2
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1094312.7
Basic Enclosure + High Efficiency Whole Home Electrification       1088524.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1064357.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1058569.4
High Efficiency Whole Home Electrification                          988661.9
High Efficiency Heat Pump with Electric Heat Backup                 771292.3
Mininum Efficiency Whole Home Electrification                       746096.4
Mininum Efficiency Heat Pump with Electric Heat Backup              580351.7
Enhanced Enclosure                                                  450198.5
Basic Enclosure                                                     398519.6
Mininum Efficiency Heat Pump with Existing Heat Backup              370849.8
Heat Pump Water Heater                                              120385.4
Induction Cooking                                              

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2573605.1
Basic Enclosure + High Efficiency Whole Home Electrification       2545097.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2494466.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2465959.3
High Efficiency Whole Home Electrification                         2331833.4
High Efficiency Heat Pump with Electric Heat Backup                1605999.9
Mininum Efficiency Whole Home Electrification                      1440823.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1114151.2
Enhanced Enclosure                                                  850366.0
Mininum Efficiency Heat Pump with Existing Heat Backup              766843.2
Basic Enclosure                                                     741454.5
Heat Pump Water Heater                                              422119.3
Induction Cooking                                              

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4738695.9
Basic Enclosure + High Efficiency Whole Home Electrification       4698402.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4608500.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4568206.5
High Efficiency Whole Home Electrification                         4267534.4
High Efficiency Heat Pump with Electric Heat Backup                3162224.0
Mininum Efficiency Whole Home Electrification                      2911421.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2277528.6
Enhanced Enclosure                                                 1775546.9
Basic Enclosure                                                    1556683.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1474505.8
Heat Pump Water Heater                                              638241.9
Induction Cooking                                              

### Annual Total Emission Savings

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    65390522.7
Basic Enclosure + High Efficiency Whole Home Electrification       64709740.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     64126687.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        63445905.3
High Efficiency Whole Home Electrification                         51427581.3
High Efficiency Heat Pump with Electric Heat Backup                40885108.0
Enhanced Enclosure                                                 32668219.5
Basic Enclosure                                                    28583734.5
Mininum Efficiency Whole Home Electrification                      27189257.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18665164.7
Mininum Efficiency Heat Pump with Existing Heat Backup             16723672.1
Heat Pump Water Heater                                              5834525.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    69610936.2
Basic Enclosure + High Efficiency Whole Home Electrification       68953037.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     67831725.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        67173826.3
High Efficiency Whole Home Electrification                         57711233.0
High Efficiency Heat Pump with Electric Heat Backup                43383205.2
Enhanced Enclosure                                                 31813987.6
Mininum Efficiency Whole Home Electrification                      31438555.7
Basic Enclosure                                                    28120349.4
Mininum Efficiency Heat Pump with Electric Heat Backup             22415816.5
Mininum Efficiency Heat Pump with Existing Heat Backup             19192462.5
Heat Pump Water Heater                                              7416318.2
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    184710974.0
Basic Enclosure + High Efficiency Whole Home Electrification       181539755.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     180084085.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        176912867.4
High Efficiency Whole Home Electrification                         157438198.2
High Efficiency Heat Pump with Electric Heat Backup                108555818.1
Enhanced Enclosure                                                  66482413.9
Mininum Efficiency Whole Home Electrification                       61052321.5
Basic Enclosure                                                     57930654.8
Mininum Efficiency Heat Pump with Electric Heat Backup              54668209.6
Mininum Efficiency Heat Pump with Existing Heat Backup              49982772.4
Heat Pump Water Heater                                              27544963.9
Induction Cooking                      

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    319712432.9
Basic Enclosure + High Efficiency Whole Home Electrification       315202533.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     312042498.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        307532599.0
High Efficiency Whole Home Electrification                         266577012.5
High Efficiency Heat Pump with Electric Heat Backup                192824131.3
Enhanced Enclosure                                                 130964620.9
Mininum Efficiency Whole Home Electrification                      119680134.8
Basic Enclosure                                                    114634738.7
Mininum Efficiency Heat Pump with Electric Heat Backup              95749190.8
Mininum Efficiency Heat Pump with Existing Heat Backup              85898907.1
Heat Pump Water Heater                                              40795807.5
Induction Cooking                      

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [49]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure                                                  498.8
Basic Enclosure                                                     431.3
Enhanced Enclosure + High Efficiency Whole Home Electrification     390.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      389.9
Basic Enclosure + High Efficiency Whole Home Electrification        360.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         360.4
Heat Pump Clothes Dryer                                              16.0
Heat Pump Water Heater                                               12.8
Induction Cooking                                                   -18.9
Electric Cooking                                                    -40.8
Electric Clothes Dryer                                              -64.4
High Efficiency Whole Home Electrification                         -219.0
Mininum Efficiency Heat Pump with Existing Heat Backup             -299.9
High Efficiency Heat Pump

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    423.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     422.1
Basic Enclosure + High Efficiency Whole Home Electrification       404.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        402.8
Enhanced Enclosure                                                 370.7
Basic Enclosure                                                    325.8
High Efficiency Whole Home Electrification                          72.1
Heat Pump Clothes Dryer                                             21.0
Heat Pump Water Heater                                              12.2
Induction Cooking                                                  -21.8
High Efficiency Heat Pump with Electric Heat Backup                -23.1
Electric Cooking                                                   -39.1
Electric Clothes Dryer                                             -58.4
Mininum Efficiency Heat Pump with Exis

segment3


upgrade_name
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     525.4
Enhanced Enclosure + High Efficiency Whole Home Electrification    522.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        503.8
Basic Enclosure + High Efficiency Whole Home Electrification       500.7
High Efficiency Whole Home Electrification                         338.7
Enhanced Enclosure                                                 258.2
High Efficiency Heat Pump with Electric Heat Backup                249.1
Basic Enclosure                                                    227.7
Mininum Efficiency Heat Pump with Existing Heat Backup              68.9
Heat Pump Water Heater                                              34.5
Heat Pump Clothes Dryer                                             22.2
Induction Cooking                                                  -14.6
Electric Cooking                                                   -40.1
Electric Clothes Dryer                

segment4


upgrade_name
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     493.2
Enhanced Enclosure + High Efficiency Whole Home Electrification    491.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        471.1
Basic Enclosure + High Efficiency Whole Home Electrification       469.1
Enhanced Enclosure                                                 304.4
Basic Enclosure                                                    267.8
High Efficiency Whole Home Electrification                         232.4
High Efficiency Heat Pump with Electric Heat Backup                141.8
Heat Pump Water Heater                                              28.3
Heat Pump Clothes Dryer                                             20.7
Mininum Efficiency Heat Pump with Existing Heat Backup             -11.8
Induction Cooking                                                  -16.3
Electric Cooking                                                   -40.0
Electric Clothes Dryer                

### Annual Energy Savings Per Dwelling Unit

In [50]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    96.1
Basic Enclosure + High Efficiency Whole Home Electrification       95.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     94.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        93.7
High Efficiency Whole Home Electrification                         85.0
High Efficiency Heat Pump with Electric Heat Backup                70.4
Mininum Efficiency Whole Home Electrification                      65.3
Mininum Efficiency Heat Pump with Electric Heat Backup             52.3
Enhanced Enclosure                                                 42.6
Basic Enclosure                                                    37.4
Mininum Efficiency Heat Pump with Existing Heat Backup             31.0
Heat Pump Water Heater                                              8.6
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    66.3
Basic Enclosure + High Efficiency Whole Home Electrification       66.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     64.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        64.1
High Efficiency Whole Home Electrification                         59.9
Mininum Efficiency Whole Home Electrification                      47.6
High Efficiency Heat Pump with Electric Heat Backup                46.7
Mininum Efficiency Heat Pump with Electric Heat Backup             35.2
Enhanced Enclosure                                                 27.3
Basic Enclosure                                                    24.1
Mininum Efficiency Heat Pump with Existing Heat Backup             22.5
Heat Pump Water Heater                                              7.3
Electric Cooking                                                    1.3
Induction Cooking                                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    35.6
Basic Enclosure + High Efficiency Whole Home Electrification       35.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     34.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34.1
High Efficiency Whole Home Electrification                         32.3
Mininum Efficiency Whole Home Electrification                      24.6
High Efficiency Heat Pump with Electric Heat Backup                22.2
Mininum Efficiency Heat Pump with Electric Heat Backup             15.4
Enhanced Enclosure                                                 12.1
Mininum Efficiency Heat Pump with Existing Heat Backup             10.9
Basic Enclosure                                                    10.7
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    47.5
Basic Enclosure + High Efficiency Whole Home Electrification       47.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     46.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        45.7
High Efficiency Whole Home Electrification                         42.7
Mininum Efficiency Whole Home Electrification                      34.1
High Efficiency Heat Pump with Electric Heat Backup                31.7
Mininum Efficiency Heat Pump with Electric Heat Backup             22.8
Enhanced Enclosure                                                 18.1
Basic Enclosure                                                    16.0
Mininum Efficiency Heat Pump with Existing Heat Backup             15.1
Heat Pump Water Heater                                              6.4
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [51]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    70.5
Basic Enclosure + High Efficiency Whole Home Electrification       70.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     69.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        68.6
High Efficiency Whole Home Electrification                         61.9
High Efficiency Heat Pump with Electric Heat Backup                50.2
Mininum Efficiency Whole Home Electrification                      45.9
Mininum Efficiency Heat Pump with Electric Heat Backup             35.7
Enhanced Enclosure                                                 31.7
Basic Enclosure                                                    27.5
Mininum Efficiency Heat Pump with Existing Heat Backup             21.6
Heat Pump Water Heater                                              7.2
Electric Cooking                                                    1.1
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    69.8
Basic Enclosure + High Efficiency Whole Home Electrification       69.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     67.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        67.4
High Efficiency Whole Home Electrification                         63.4
Mininum Efficiency Whole Home Electrification                      49.6
High Efficiency Heat Pump with Electric Heat Backup                47.7
Mininum Efficiency Heat Pump with Electric Heat Backup             35.3
Enhanced Enclosure                                                 28.4
Basic Enclosure                                                    25.1
Mininum Efficiency Heat Pump with Existing Heat Backup             21.8
Heat Pump Water Heater                                              9.2
Electric Cooking                                                    1.5
Induction Cooking                                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    62.2
Basic Enclosure + High Efficiency Whole Home Electrification       61.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     60.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        59.5
High Efficiency Whole Home Electrification                         56.8
Mininum Efficiency Whole Home Electrification                      39.1
High Efficiency Heat Pump with Electric Heat Backup                36.9
Mininum Efficiency Heat Pump with Electric Heat Backup             24.5
Enhanced Enclosure                                                 19.7
Mininum Efficiency Heat Pump with Existing Heat Backup             18.0
Basic Enclosure                                                    17.6
Heat Pump Water Heater                                             12.2
Electric Cooking                                                    2.4
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.4
Basic Enclosure + High Efficiency Whole Home Electrification       63.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     62.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        61.8
High Efficiency Whole Home Electrification                         58.5
Mininum Efficiency Whole Home Electrification                      41.9
High Efficiency Heat Pump with Electric Heat Backup                40.2
Mininum Efficiency Heat Pump with Electric Heat Backup             27.5
Enhanced Enclosure                                                 22.5
Basic Enclosure                                                    20.0
Mininum Efficiency Heat Pump with Existing Heat Backup             19.0
Heat Pump Water Heater                                             11.1
Electric Cooking                                                    2.0
Induction Cooking                                  

### Annual Emission Savings Per Dwelling Unit

In [52]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5868.9
Basic Enclosure + High Efficiency Whole Home Electrification       5807.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5755.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5694.3
High Efficiency Whole Home Electrification                         4615.7
High Efficiency Heat Pump with Electric Heat Backup                3669.5
Enhanced Enclosure                                                 2932.0
Basic Enclosure                                                    2565.4
Mininum Efficiency Whole Home Electrification                      2450.5
Mininum Efficiency Heat Pump with Electric Heat Backup             1675.2
Mininum Efficiency Heat Pump with Existing Heat Backup             1539.6
Heat Pump Water Heater                                              523.7
Heat Pump Clothes Dryer                                              72.0
Electric Cooking         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4218.0
Basic Enclosure + High Efficiency Whole Home Electrification       4178.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4110.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4070.4
High Efficiency Whole Home Electrification                         3497.0
High Efficiency Heat Pump with Electric Heat Backup                2628.8
Mininum Efficiency Whole Home Electrification                      2007.1
Enhanced Enclosure                                                 1927.8
Basic Enclosure                                                    1703.9
Mininum Efficiency Heat Pump with Electric Heat Backup             1358.3
Mininum Efficiency Heat Pump with Existing Heat Backup             1166.3
Heat Pump Water Heater                                              451.9
Heat Pump Clothes Dryer                                              66.1
Electric Cooking         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2557.9
Basic Enclosure + High Efficiency Whole Home Electrification       2514.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2493.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2449.9
High Efficiency Whole Home Electrification                         2180.2
High Efficiency Heat Pump with Electric Heat Backup                1503.3
Mininum Efficiency Whole Home Electrification                      1041.8
Enhanced Enclosure                                                  943.8
Basic Enclosure                                                     834.0
Mininum Efficiency Heat Pump with Electric Heat Backup              757.0
Mininum Efficiency Heat Pump with Existing Heat Backup              708.2
Heat Pump Water Heater                                              384.7
Heat Pump Clothes Dryer                                              64.6
Electric Cooking         

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3201.7
Basic Enclosure + High Efficiency Whole Home Electrification       3156.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3124.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3079.7
High Efficiency Whole Home Electrification                         2669.6
High Efficiency Heat Pump with Electric Heat Backup                1931.0
Mininum Efficiency Whole Home Electrification                      1402.1
Enhanced Enclosure                                                 1335.2
Basic Enclosure                                                    1180.5
Mininum Efficiency Heat Pump with Electric Heat Backup              958.9
Mininum Efficiency Heat Pump with Existing Heat Backup              877.4
Heat Pump Water Heater                                              411.4
Heat Pump Clothes Dryer                                              66.5
Electric Cooking         

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [53]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [54]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [55]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [56]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [57]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_61111/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_61111/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_61111/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure,15.5,12.4
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.5,12.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.5,12.6
Basic Enclosure + High Efficiency Whole Home Electrification,15.5,12.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.5,12.7
Basic Enclosure,15.5,12.8
Heat Pump Clothes Dryer,15.1,14.9
Heat Pump Water Heater,15.5,15.5
Induction Cooking,15.3,15.4


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.3,12.0
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.3,12.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.3,12.1
Basic Enclosure + High Efficiency Whole Home Electrification,15.3,12.2
Enhanced Enclosure,15.3,12.4
Basic Enclosure,15.3,12.8
High Efficiency Whole Home Electrification,15.3,14.7
Heat Pump Clothes Dryer,15.1,15.0
Heat Pump Water Heater,15.3,15.2


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,16.1,10.7
Enhanced Enclosure + High Efficiency Whole Home Electrification,16.1,10.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,16.1,10.9
Basic Enclosure + High Efficiency Whole Home Electrification,16.1,11.0
High Efficiency Whole Home Electrification,16.1,12.7
Enhanced Enclosure,16.2,13.6
High Efficiency Heat Pump with Electric Heat Backup,16.1,13.6
Basic Enclosure,16.2,13.9
Mininum Efficiency Heat Pump with Existing Heat Backup,16.3,15.7


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.9,11.1
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.9,11.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.9,11.3
Basic Enclosure + High Efficiency Whole Home Electrification,15.9,11.4
Enhanced Enclosure,15.9,13.3
High Efficiency Whole Home Electrification,15.9,13.4
Basic Enclosure,16.0,13.6
High Efficiency Heat Pump with Electric Heat Backup,15.9,14.2
Heat Pump Water Heater,15.8,15.6


In [58]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [59]:
! jupyter nbconvert --output-dir='./results/hill_district/' --to=html --no-input saving_results_reports_for_hill_district.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_hill_district.ipynb to html
/Users/lliu2/micromamba/envs/res-est/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 356710 bytes to results/hill_district/saving_results_reports_for_hill_district.html
